In [141]:
import pandas as pd
import numpy as np
from itertools import groupby
from tqdm import tqdm

In [4]:
d_ms = pd.read_csv('../example_data/clustering/sample0815.csv')

In [5]:
d_ms.head()

,Average Rt(min),Average Mz,S/N average,20180815_CEC_CAL-8-no4_MSpos_1,20180815_CEC_CAL-8-no4_MSpos_2,20180815_CEC_CAL-8-no4_MSpos_3,20180815_CEC_CAL-8-no4_MSpos_4,20180815_CEC_CAL-8-no4_MSpos_5,20180815_CEC_CAL-8-no4_MSpos_6,20180815_CEC_CAL-8-no4_MSpos_7,...,20180815_SR520-Creek_Mix6A_3,20180815_SR520-Creek_Mix6B_1,20180815_SR520-Creek_Mix6B_2,20180815_SR520-Creek_Mix6B_3,20180815_Swan-Creek-Dec_1,20180815_Swan-Creek-Dec_2,20180815_Swan-Creek-Dec_3,20180815_Swan-Creek-May_1,20180815_Swan-Creek-May_2,20180815_Swan-Creek-May_3
0,8.273,100.03867,40.44,0,0,88,0,0,89,0,...,238,113,184,202,0,0,0,0,76,0
1,8.304,100.03956,77.03,0,0,88,0,0,88,0,...,238,217,184,202,0,0,0,0,73,0
2,7.097,100.07603,52.53,64,231,104,247,259,145,294,...,1375,1139,511,1159,1201,1337,1723,573,2585,1585
3,5.226,100.11173,31.39,1629,882,1850,2217,1254,946,1095,...,18206,20583,20713,19942,21409,21764,23078,18056,18226,17470
4,4.944,100.11175,14.70,759,934,396,1676,583,639,336,...,10508,8889,14385,6274,12796,14584,13262,7576,12092,11060


In [6]:
d_ms = d_ms.rename(columns={'Average Rt(min)': 'Average RT (min)', 'Average Mz': 'Average m/z', 'S/N average': 'Average sn'})
d_ms.insert(3, "Average score", 1)

In [7]:
def d_clean(dataframe, rt_range=[0, 30], mz_range=[0, 1200], sn_thres=3, score_thres=0, area_thres=5000): #Update with *args or **args in future updates
    #Area thres update
    drop_index = np.argwhere(np.asarray(dataframe[dataframe.columns[4:]].max(axis=1)) < area_thres).reshape(1,-1)
    df_c = dataframe.drop(drop_index[0])
    
    df_c = df_c[(df_c['Average RT (min)'] > rt_range[0]) & (df_c['Average RT (min)'] < rt_range[1])]
    df_c = df_c[(df_c['Average m/z'] > mz_range[0]) & (df_c['Average m/z'] < mz_range[1])]
    df_c = df_c[df_c['Average sn'] >= sn_thres]
    df_c = df_c[df_c['Average score'] >= score_thres]
    df_c.reset_index(inplace=True)
    df_c.drop(columns=['index'],inplace=True)
    
    return df_c

In [75]:
df_c = d_clean(d_ms,rt_range = [1,30], mz_range = [200,800], area_thres=500)

In [76]:
#Distinguish between sample and blank
col_blank = [col for col in df_c.columns if 'CEC' in col or 'Blank' in col or 'ISTD' in col or 'Wash' in col or 'Shutdown' in col]
col_sample = [col for col in df_c.columns if col not in col_blank]

In [143]:
#sample vs blank threshold
samplevsblank_thres = 10
d_sample = df_c[df_c[col_sample[4:]].max(axis=1) / df_c[col_drop].mean(axis=1) > samplevsblank_thres][col_sample]
d_sample.reset_index(inplace=True)
d_sample.drop(columns=['index'],inplace=True)

In [144]:
#Noise removal from triplicates
trip_list = [list(i) for j, i in groupby(d_sample.columns[4:], lambda a: a.split('_')[1])] #Needs to define 1. parser 2. position of parsing

In [145]:
empty_tol = 0
mvm_tol = 10

for triplicate in tqdm(trip_list):
    for index, row in d_sample[triplicate].iterrows():
        if (row == 0).sum() > empty_tol:
            d_sample.loc[index, triplicate] = 0
            #Filling the gaps and check variance?
#         elif row.max() / row.mean() > mvm_tol:
#             d_sample.loc[index, triplicate] = 0

100%|██████████| 9/9 [02:17<00:00, 15.33s/it]


In [172]:
#Double check if any empty columns and clean up
d_sample = d_sample[(d_sample.iloc[:,4:]!=0).sum(1) > 3]

# whole batch or only dilution series?

In [175]:
d_sample.head()

,Average RT (min),Average m/z,Average sn,Average score,20180815_Coulter-Creek-May_1,20180815_Coulter-Creek-May_2,20180815_Coulter-Creek-May_3,20180815_Crescent-Creek-Jan_1,20180815_Crescent-Creek-Jan_2,20180815_Crescent-Creek-Jan_3,...,20180815_SR520-Creek_Mix6A_3,20180815_SR520-Creek_Mix6B_1,20180815_SR520-Creek_Mix6B_2,20180815_SR520-Creek_Mix6B_3,20180815_Swan-Creek-Dec_1,20180815_Swan-Creek-Dec_2,20180815_Swan-Creek-Dec_3,20180815_Swan-Creek-May_1,20180815_Swan-Creek-May_2,20180815_Swan-Creek-May_3
0,8.381,200.04636,9.85,1,5520,11208,5499,30277,6087,12761,...,11867,16143,9538,7513,12569,13390,16329,3868,7647,3711
1,8.326,200.04755,17.23,1,31435,10739,43573,27776,10074,22430,...,13405,16143,15061,16500,9351,9545,18518,6118,10946,5523
2,5.630,200.07532,32.75,1,1772,1193,1470,1635,1258,3545,...,2272,1755,1478,2092,2444,1473,472,710,979,581
3,5.718,200.12758,45.73,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.170,200.12761,18.43,1,3256,3790,948,2253,1292,949,...,962,3579,3137,1880,1281,2800,2111,1005,1172,1114
